In [100]:
class FirstClass: # Define a class object
    def setdata(self, value): # Define class's methods
        self.data = value # self is the instance
    def display(self):
        print(self.data)

class SecondClass(FirstClass): # Inherits setdata
    def display(self): # Changes display
        print('Current value = "%s"' % self.data)

class ThirdClass(SecondClass): # Inherit from SecondClass
    def __init__(self, value): # On "ThirdClass(value)"
        self.data = value
    def __add__(self, other): # On "self + other"
        return ThirdClass(self.data + other)
    def __str__(self): # On "print(self)", "str()"
        return f'[ThirdClass: {self.data}]'
    def mul(self,count):
        self.data *= count
        

a = ThirdClass('abc') # __init__ called
a.display()
print(a)
a.mul(3) # mul: changes instance in place
print(a)


Current value = "abc"
[ThirdClass: abc]
[ThirdClass: abcabcabc]


Assign a Chromosome class

In [101]:
import pandas as pd
import re
import os

class Chromosome:
    count = 0
    def __init__(self, filename):  #initilize filename
        self.filename = filename
        Chromosome.count+=1



        def get_data(filename):  #get csv data
            data = pd.read_csv("data\\syntenic\\" + filename, skiprows=[0])
            return data

        def get_species(filename):  #get species name by filename
            species = re.search("[A-Za-z\s]+", filename)
            return species.group(0)

        def get_chromosome_location(filename):  #get chromosome_location by filename
            temp = re.sub(".CSV", "", filename)  #remove filetype
            location = re.search("[_]([A-Za-z0-9\s_.]+)", temp)
            return location.group(1)

        def get_gene(data):  #get gene
            neighbourgenes = []
            for gene in data["Gene symbol"]:
                if "CACNG" in gene.upper():
                    neighbourgenes.append(gene)
                elif "TMEM114" in gene.upper():
                    neighbourgenes.append(gene)
                elif "TMEM235" in gene.upper():
                    neighbourgenes.append(gene)
            return neighbourgenes

        def get_neighbour_genes(genes,data,n):
            neighbourindex = []  #a new list to save index of neighbour genes
            neighbourgene = []
            for index, gene in enumerate(data["Gene symbol"]):  #a loop to save neighbourgene index
                if gene in genes:
                    for i in range (index-n, index+n+1):
                        neighbourindex.append(i)
            neighbourindex = list(set(neighbourindex))
            for i in neighbourindex:
                try:
                    neighbourgene.append(data["Gene symbol"][i])
                except:
                    pass
            return neighbourgene

        self.data = get_data(self.filename)
        self.species = get_species(self.filename)
        self.location = get_chromosome_location(self.filename)
        self.targetgenes = get_gene(self.data)
        self.neighbour = get_neighbour_genes(self.targetgenes,self.data,5)
        self.textx = None
        self.texty = None

    def set_genes(self,gene):
        self.genes = gene


Initialize class

In [102]:
filelist = os.listdir("data\\syntenic")  #get file list
for index, file in enumerate(filelist):  #a loop to initialize class chromosome
    exec(f"c{index} = Chromosome(file)")  #initialize chromosome
    exec('print(f"c{index}: ", end = "")')  #print initialized class name
    exec(f"print(c{index}.species, c{index}.location, c{index}.neighbour)") #print species and location
    print()

c0: Anolis carolinensis C2 ['pgs1', 'LOC103277795', 'socs3', 'LOC103277796', 'LOC100564571', 'tmem235', 'birc5', 'afmid', 'tk1', 'syngr2', 'tmc8']

c1: Anolis carolinensis C5 ['myh9', 'LOC103278846', 'txn2', 'foxred2', 'eif3d', 'cacng2', 'ift27', 'pvalb', 'ncf4', 'csf2rb', 'tex33']

c2: Anolis carolinensis scaffold NW_003338797.1 ['cacng5', 'cacng4', 'cacng1', 'helz', 'psmd12', 'pitpnc1', 'LOC107983219', 'nol11']

c3: Anolis carolinensis scaffold NW_003338810.1 ['rbfox1', 'mir5411', 'LOC107983238', 'LOC107983239', 'tmem114', 'mettl22', 'abat', 'tmem186', 'pmm2', 'carhsp1']

c4: Anolis carolinensis scaffold NW_003338876.1 ['ccdc114', 'myadm', 'LOC100557974', 'prkcg', 'LOC103280215', 'cacng6', 'cacng8', 'cacng7', 'LOC100560082', 'rps9', 'lmtk3', 'LOC100560478', 'LOC100560675']

c5: Canis lupus familiaris C1 ['LOC119870904', 'LOC484314', 'VSTM1', 'CACNG6', 'CACNG8', 'CACNG7', 'PRKCG', 'MYADM', 'NLRP12', 'MIR371', 'LOC106559259', 'LOC106559612', 'LOC119875990']

c6: Canis lupus familiaris 

Create dataframe

In [103]:
df = pd.DataFrame()

for index, file in enumerate(filelist):  #add column name species and location to class.data (Dataframe)
     exec(f"df{index} = c{index}.data")  #assign df 
     exec(f"df{index}.insert(0,'Species', c{index}.species)")  #insert species
     exec(f"df{index}.insert(1,'Location', c{index}.location)")  #insert location
     exec(f"df = df.append(df{index}, ignore_index=True)")

df['Gene symbol'] = df['Gene symbol'].str.upper()  #change gene symbol to upper letter

Set distance

In [104]:
distance = 10  #distance as neighbour
neighbourindex = []  #a new list to save index of neighbour genes
for index, gene in enumerate(df["Gene symbol"]):  #a loop to save neighbourgene index
    if ("CACNG") in gene:
        for i in range (index-distance, index+distance+1):
            neighbourindex.append(i)
    elif ("TMEM114") in gene:
        for i in range (index-distance, index+distance+1):
            neighbourindex.append(i)
    elif ("TMEM235") in gene:
        for i in range (index-distance, index+distance+1):
            neighbourindex.append(i)

neighbourindex = sorted(list(set(neighbourindex)))  #remove duplicates
newdf = df.iloc[neighbourindex, :-1]  #a new data by neighbourindex and remove name column

In [105]:
scoredict = {}  #a dictionary to store score
for gene in newdf["Gene symbol"]:  #loop to calculate score
    if "CACNG" in gene:
        scoredict[gene] = 100
    elif "TMEM114" in gene:
        scoredict[gene] = 100
    elif "TMEM235" in gene:
        scoredict[gene] = 100
    else:
        try:
            scoredict[gene]
            scoredict[gene].append(1)  #add score
        except:
            scoredict[gene] = [1]  #build score

for gene in scoredict:  #append list item to score
    if type(scoredict[gene]) == list:
        scoredict[gene] = len(scoredict[gene])


Set a minimum score

In [106]:
score = 2  ###set a minimum score
genelist = []  #gene that higher than score
for gene in scoredict: 
    if scoredict[gene] >= score:
        genelist.append(gene)


for index in range (Chromosome.count):  #remove each neighbourgene that not in genelist(score lower than n)
    templist = []
    exec(f"""
for neighbourgene in c{index}.neighbour:
    if neighbourgene.upper() in genelist:
        templist.append(neighbourgene.upper())""")  #append score that is higher than n to a templist
    exec(f"c{index}.set_genes(templist)")  #set Chromosome.gene as templist
    exec(f'print("c{index}: ", end="")')  #print 
    exec(f"print(c{index}.species, c{index}.location, c{index}.genes)") #print species and location
    print()

c0: Anolis carolinensis C2 ['PGS1', 'SOCS3', 'TMEM235', 'BIRC5', 'AFMID', 'TK1', 'SYNGR2', 'TMC8']

c1: Anolis carolinensis C5 ['MYH9', 'TXN2', 'FOXRED2', 'EIF3D', 'CACNG2', 'IFT27', 'PVALB', 'NCF4', 'CSF2RB', 'TEX33']

c2: Anolis carolinensis scaffold NW_003338797.1 ['CACNG5', 'CACNG4', 'CACNG1', 'HELZ', 'PSMD12', 'PITPNC1', 'NOL11']

c3: Anolis carolinensis scaffold NW_003338810.1 ['RBFOX1', 'TMEM114', 'METTL22', 'ABAT', 'TMEM186', 'PMM2', 'CARHSP1']

c4: Anolis carolinensis scaffold NW_003338876.1 ['MYADM', 'PRKCG', 'CACNG6', 'CACNG8', 'CACNG7', 'RPS9']

c5: Canis lupus familiaris C1 ['VSTM1', 'CACNG6', 'CACNG8', 'CACNG7', 'PRKCG', 'MYADM', 'NLRP12']

c6: Canis lupus familiaris C10 ['CACNG2', 'EIF3D', 'FOXRED2', 'CSF2RB', 'NCF4', 'PVALB', 'IFT27']

c7: Canis lupus familiaris C6 ['PMM2', 'TMEM186', 'ABAT', 'METTL22', 'TMEM114', 'RBFOX1', 'RBBP6', 'CACNG3', 'PRKCB', 'CHP2']

c8: Canis lupus familiaris C9 ['PGS1', 'SOCS3', 'TMEM235', 'BIRC5', 'AFMID', 'TK1', 'SYNGR2', 'PSMD12', 'HELZ',

In [107]:
specieslist = set()
for index in range(Chromosome.count):
    exec(f'specieslist.add(c{index}.species)')
specieslist = list(specieslist)

Initialize draw location

In [108]:
buildxy = """
initialpoint = 300
tempx = initialpoint
tempy = initialpoint
step = 80
species = c0.species
c0.textx = initialpoint
c0.texty = initialpoint
for index in range(1,Chromosome.count):
    exec(f'''
if c{index}.species == species:
    c{index}.textx = tempx
    c{index}.texty = c{index-1}.texty + step*len(c{index-1}.genes) + 300
else:
    tempx += 1000
    species = c{index}.species
    c{index}.texty = initialpoint
    c{index}.textx = tempx

#print(c{index}.textx,c{index}.texty)''') 
"""

exec(buildxy)

Draw Image

In [109]:
draw = """
from PIL import Image, ImageDraw, ImageFont

exec(f"newImage = Image.new('RGBA', (c{Chromosome.count-1}.textx + 800,10000), 'White')")
drawObj = ImageDraw.Draw(newImage)

fontInfo1 = ImageFont.truetype('C:\Windows\Fonts\calibri.ttf', 80)
fontInfo2 = ImageFont.truetype('C:\Windows\Fonts\calibri.ttf', 50)

for i in range(Chromosome.count):
    exec(f'drawObj.text((c{i}.textx, c{i}.texty), c{i}.location, fill="black", font=fontInfo2)')
    exec(f'''
ladder = 0
step = 80
for gene in c{i}.genes:
    if c{i}.texty == initialpoint:
        drawObj.text((c{i}.textx, 50), c{i}.species, fill="black",font=fontInfo1)
    ladder += step
    if "CACNG" in gene:
        drawObj.text((c{i}.textx, c{i}.texty + ladder), gene, fill="red", font=fontInfo2)
    elif "TMEM114" in gene:
        drawObj.text((c{i}.textx, c{i}.texty + ladder), gene, fill="red", font=fontInfo2)
    elif "TMEM235" in gene:
        drawObj.text((c{i}.textx, c{i}.texty + ladder), gene, fill="red", font=fontInfo2)
    else:
        drawObj.text((c{i}.textx, c{i}.texty + ladder), gene, fill="black", font=fontInfo2)
    ''')
newImage.show()
"""

exec(draw)

Swap

In [110]:
# def Swap():
#     x,y = input().split()
#     cmd = f"{x}, {y} = {y}, {x}"
#     return cmd

# exec(Swap())
# exec(buildxy)
# exec(draw)